In [ ]:
from lenskit.datasets import ML100K
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms.als import ImplicitMF
from lenskit.algorithms.svd import BiasedSVD
from lenskit.algorithms.funksvd import FunkSVD
from lenskit.algorithms import Recommender, als, item_knn, user_knn
from lenskit.algorithms.basic import Popular, Bias
from lenskit import topn
import pandas as pd
%matplotlib inline
import scipy.sparse as sp
import numpy as np
import pickle 

In [ ]:
## Load Scipy Matrices
ratingAll = sp.load_npz('UserItemMatrix.npz')
ratingTrain = sp.load_npz('TrainUserItemMatrix.npz')
ratingValid = sp.load_npz('ValidationUserItemMatrix.npz')
ratingTest = sp.load_npz('TestUserItemMatrix.npz')

## Load the LookUp Dictionaries
songToIndex = pickle.load(open('songIdToIndex.p','rb'))
songIndexToId = pickle.load(open('songIndexToId.p','rb'))
DFLookUp = pd.DataFrame.from_dict(songIndexToId, orient='index')

In [ ]:
## Create Train, Validation and Test from the Scipy Matrices in DF format with columns user, item and rating
listOfListsTrain = []
for i in range(ratingTrain.shape[0]-900000):
    for j in ratingTrain[i].nonzero()[1].tolist():
        listOfListsTrain.append([i, j])
train = pd.DataFrame(listOfListsTrain, columns=['user', 'item'])
train['rating'] = 1

listOfListsValidation = []
for i in range(ratingValid.shape[0]-990000):
    for j in ratingTest[i].nonzero()[1].tolist():
        listOfListsValidation.append([i, j])
validation = pd.DataFrame(listOfListsValidation, columns=['user', 'item'])
validation['rating'] = 1

listOfListsTest = []
for i in range(ratingTest.shape[0]-990000):
    for j in ratingTest[i].nonzero()[1].tolist():
        listOfListsTest.append([i, j])
test = pd.DataFrame(listOfListsTest, columns=['user', 'item'])
test['rating'] = 1




In [ ]:
## Fit the model

algo = Recommender.adapt(ImplicitMF(features=4)) # Required by Lenskit Library
algo.fit(train) # Fit The model


In [ ]:
# Get recommendations for a single user
rec_u_i = algo.recommend(train['user'].unique()[2], 1)
#rec_u_i = algo.predict_for_user(train['user'].unique()[2], [1,2])
rec_u_i

In [ ]:
## Map Recommendation Index to Song ID and retrieve meta data from DFLookUp
user_i_playlist_recommendations = []
for i in rec_u_i['item'].tolist():
    user_i_playlist_recommendations.append(DFLookUp.loc[i].to_dict())

In [ ]:
user_i_playlist_recommendations

In [ ]:
# Run through all users to get recommendations
recs = {}
for user in train['user'].unique(): 
    user_recs = algo.recommend(user, 2)
    recs[user] = user_recs

In [ ]:
## Map Recommendation Index to Song ID and retrieve meta data from DFLookUp
user_i_playlist_recommendations = []
for i in rec_u_i['item'].tolist():
    user_i_playlist_recommendations.append(DFLookUp.loc[i].to_dict())

In [ ]:
# Evaluation
# evaluate the recommender
print(topn.precision(rec_u_i, train[train['user']==0], k=10))